In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency, norm
from data_viz import *

In [2]:
data = pd.read_csv("data/animal_center.csv")
data

,animal_id,name,animal_type,sex_intake,sex_outcome,breed,color,date_of_birth,found_location,intake_datetime,outcome_datetime,intake_type,intake_condition,outcome_type,outcome_subtype,age_upon_intake(years),age_upon_outcome(years),duration(days)
0,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research in Austin (TX),2014-03-07 14:26:00,2014-03-08 17:10:00,Public Assist,Normal,Return to Owner,NaN,6.7,6.7,1.0
1,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research Blvd in Austin (TX),2014-12-19 10:21:00,2014-12-20 16:35:00,Public Assist,Normal,Return to Owner,NaN,7.4,7.4,1.0
2,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,Colony Creek And Hunters Trace in Austin (TX),2017-12-07 14:07:00,2017-12-07 00:00:00,Stray,Normal,Return to Owner,NaN,10.4,10.4,-1.0
3,A047759,Oreo,Dog,Neutered Male,Neutered Male,Dachshund,Tricolor,2004-04-02,Austin (TX),2014-04-02 15:55:00,2014-04-07 15:12:00,Owner Surrender,Normal,Transfer,Partner,10.0,10.0,4.0
4,A134067,Bandit,Dog,Neutered Male,Neutered Male,Shetland Sheepdog,Brown/White,1997-10-16,12034 Research Blvd in Austin (TX),2013-11-16 09:02:00,2013-11-16 11:54:00,Public Assist,Injured,Return to Owner,NaN,16.1,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157585,A894036,NaN,Dog,Intact Female,NaN,Miniature Pinscher Mix,Black/Tan,NaN,Martin Luther King Jr Blvd And M Sta Ln in Aus...,2023-12-01 17:33:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157586,A894054,Dawn,Dog,Intact Female,Intact Female,Chihuahua Shorthair,Tan,2022-12-02,2001 Ponciana Loop in Austin (TX),2023-12-02 09:52:00,2023-12-02 13:03:00,Stray,Normal,Rto-Adopt,NaN,1.0,1.0,0.0
157587,A894063,Logan,Cat,Neutered Male,NaN,Domestic Shorthair,Brown Tabby/White,NaN,8585 Scotland Well Drive in Austin (TX),2023-12-02 12:49:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157588,A894067,Maverick,Dog,Intact Male,NaN,Siberian Husky,Black/White,NaN,124 West Anderson Lane in Austin (TX),2023-12-02 13:53:00,NaN,Abandoned,Normal,NaN,NaN,NaN,NaN,NaN


# **Hypothesis2:**
Has there been a significant change in the adoption rate of animals before and after the onset of COVID-19?

$H0$: There is no significant difference in the adoption rate of animals before and after the onset of COVID.\
$H1$: There is a significant increase in the adoption rate of animals after the onset of COVID compared to before.

In [3]:
adoption_research = data[["animal_id", "outcome_datetime", "outcome_type"]].reset_index(drop=True)
adoption_research = adoption_research.dropna(subset=["outcome_datetime"])
adoption_research

,animal_id,outcome_datetime,outcome_type
0,A006100,2014-03-08 17:10:00,Return to Owner
1,A006100,2014-12-20 16:35:00,Return to Owner
2,A006100,2017-12-07 00:00:00,Return to Owner
3,A047759,2014-04-07 15:12:00,Transfer
4,A134067,2013-11-16 11:54:00,Return to Owner
...,...,...,...
157563,A894009,2023-12-01 16:00:00,Transfer
157564,A894010,2023-12-01 16:00:00,Transfer
157565,A894011,2023-12-01 16:00:00,Transfer
157566,A894012,2023-12-01 16:00:00,Transfer


In [4]:
adoption_research["adoption_status"] = np.where(adoption_research["outcome_type"] == "Adoption", 1, 0)
adoption_research

,animal_id,outcome_datetime,outcome_type,adoption_status
0,A006100,2014-03-08 17:10:00,Return to Owner,0
1,A006100,2014-12-20 16:35:00,Return to Owner,0
2,A006100,2017-12-07 00:00:00,Return to Owner,0
3,A047759,2014-04-07 15:12:00,Transfer,0
4,A134067,2013-11-16 11:54:00,Return to Owner,0
...,...,...,...,...
157563,A894009,2023-12-01 16:00:00,Transfer,0
157564,A894010,2023-12-01 16:00:00,Transfer,0
157565,A894011,2023-12-01 16:00:00,Transfer,0
157566,A894012,2023-12-01 16:00:00,Transfer,0


In [5]:
datetime_threshold = "2020-01-01"
adoption_research["before_or_after_covid"] = np.where(adoption_research["outcome_datetime"] < datetime_threshold, "pre-covid", "post-covid")
adoption_research

,animal_id,outcome_datetime,outcome_type,adoption_status,before_or_after_covid
0,A006100,2014-03-08 17:10:00,Return to Owner,0,pre-covid
1,A006100,2014-12-20 16:35:00,Return to Owner,0,pre-covid
2,A006100,2017-12-07 00:00:00,Return to Owner,0,pre-covid
3,A047759,2014-04-07 15:12:00,Transfer,0,pre-covid
4,A134067,2013-11-16 11:54:00,Return to Owner,0,pre-covid
...,...,...,...,...,...
157563,A894009,2023-12-01 16:00:00,Transfer,0,post-covid
157564,A894010,2023-12-01 16:00:00,Transfer,0,post-covid
157565,A894011,2023-12-01 16:00:00,Transfer,0,post-covid
157566,A894012,2023-12-01 16:00:00,Transfer,0,post-covid


In [6]:
adoption_contingency_table = pd.crosstab(adoption_research["before_or_after_covid"], adoption_research["adoption_status"])
adoption_contingency_table

adoption_status,0,1
before_or_after_covid,,
post-covid,19404,24567
pre-covid,63341,49281


In [7]:
covid_list = ["post-covid", "pre-covid"]

for covid in covid_list:
    total = adoption_contingency_table.loc[covid].sum()
    adoption_count = adoption_contingency_table.loc[covid, 1]
    adoption_rate = round(adoption_count/total, 4)*100.0
    print(f"The adoption rate {covid} is {adoption_rate}%")

The adoption rate post-covid is 55.87%
The adoption rate pre-covid is 43.76%


In [8]:
chi2, p, dof, expected = chi2_contingency(adoption_contingency_table)
print(f"chi2:    {chi2}")
print(f"p-value: {p}")
print(f"dof:     {dof}")
print(f"expected:\n{expected}")

chi2:    1861.5467894661504
p-value: 0.0
dof:     1
expected:
[[23234.62986851 20736.37013149]
 [59510.37013149 53111.62986851]]


In [9]:
# set significance level = 0.05
alpha = 0.05
if p < alpha:
    print("There is a significant increase in the adoption rate post-COVID compared to pre-COVID.")
else:
    print("There is no significant increase in the adoption rate post-COVID compared to pre-COVID.")

There is a significant increase in the adoption rate post-COVID compared to pre-COVID.
